# Get Plaintext Dataset

In [13]:
import nltk

In [24]:
nltk.download('genesis')

[nltk_data] Downloading package genesis to
[nltk_data]     C:\Users\rosha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\genesis.zip.


True

In [25]:
from nltk.corpus import genesis

In [42]:
genesis_words = genesis.words()
filtered_words = [word for word in genesis_words if word.isalpha()]

print(len(filtered_words))
print(filtered_words[:10])

260396
['In', 'the', 'beginning', 'God', 'created', 'the', 'heaven', 'and', 'the', 'earth']


In [53]:
def make_blocks(word_list):
    word_str = "".join(word_list)
    blocked_arr = []
    
    for i in range(0, len(word_str), 5):
        block = word_str[i:i+5]

        if len(block) == 5:
            blocked_arr.append(block)
        else:
            diff = 5 - len(block)
            padded_block = block + "x" * diff
            blocked_arr.append(padded_block)

    return blocked_arr

In [72]:
pt_blocks = make_blocks(filtered_words)

print(pt_blocks[:10])

['Inthe', 'begin', 'ningG', 'odcre', 'atedt', 'hehea', 'venan', 'dthee', 'arthA', 'ndthe']


# Create Plaintext-Ciphertext dataset

In [ ]:
import random

In [85]:
def caesar_encrypt(text, key):
    result = ""

    # traverse text
    for i in range(len(text)):
        char = text[i]

        # Encrypt uppercase characters
        if (char.isupper()):
            result += chr((ord(char) + key - 65) % 26 + 65)

        # Encrypt lowercase characters
        else:
            result += chr((ord(char) + key - 97) % 26 + 97)

    return result

In [87]:
cipher = caesar_encrypt("Eesha ne mera dil Charaya", key=11)
print(cipher)

PpdslyypyxpclyotwyNslcljl


In [124]:
#This function now returns [[pt, ct, key], [pt, ct, key]]
#instead of [[pt, pt, pt], [ct, ct, ct], key] which is big yikes when training

#The function now also generates a random key for each word

def pt_ct_dataset(pt_list):
    final_list = []

    for pt in pt_list:
        key = random.randint(1, 25)
        ct = caesar_encrypt(pt, key)
        final_list.append([pt, ct, key])

    return final_list

In [118]:
def let2num(letter):
    return ord(letter)

def encode(str_list):
    encoded_list = []

    for word in str_list:
        encoded_char_list = []
        for char in word:
            encoded_char_list.append(ord(char))

        encoded_list.append(encoded_char_list)

    return encoded_list

In [131]:
# [[pt, ct, key], [pt, ct, key]]
def encode(pt_ct_key_list):
    encoded_list = []

    for pairs in pt_ct_key_list:
        encoded_pt_list = []
        encoded_ct_list = []
        
        for char in pairs[0]:
            encoded_pt_list.append(ord(char))

        for char in pairs[1]:
            encoded_ct_list.append(ord(char))

        encoded_list.append([encoded_pt_list, encoded_ct_list, pairs[2]])
    
    return encoded_list

In [130]:
dataset = pt_ct_dataset(pt_blocks)

print(dataset[:5])

[['Inthe', 'Zekyv', 17], ['begin', 'mprty', 11], ['ningG', 'rmrkK', 4], ['odcre', 'ujixk', 6], ['atedt', 'halka', 7]]


In [134]:
encoded_dataset = encode(dataset)

print(encoded_dataset[:2])

[[[73, 110, 116, 104, 101], [90, 101, 107, 121, 118], 17], [[98, 101, 103, 105, 110], [109, 112, 114, 116, 121], 11]]
